In [6]:
import pandas as pd
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.ERC_new_diapause_nondia/2.Non_diapause"
csv_file = pd.read_csv(f"{location}/ERC_correlation_nondiapause.csv")

csv_file.index = csv_file.columns
string_to_remove = "_nostop.fas.aligned.fas.best"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)

string_to_remove = ".fas.ClipKIT"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)


matrix_long = csv_file.reset_index().melt(
    id_vars="index",         # Keeps row names as a column
    var_name="Gene1",       # New name for columns
    value_name="Gene2"       # New name for matrix values
)

# Rename 'index' to 'Row' for clarity
diapause = matrix_long.rename(columns={"index": "Row"})


In [7]:
import pandas as pd
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.ERC_new_diapause_nondia/3.For Clusturing"

# diapause = pd.read_csv(f"{location}/ERC_correlation_diapause_long.csv")
# location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
gene_anno_dic = {}
with open(f"{location}/correlation_2_with_annotation.csv", 'r') as cor_file:
    for lines in cor_file.readlines()[1:]:
        gene_name = lines.split(",")[0].split("_")[0][1:]
        anno_name = lines.split(",")[-1].replace('"', '')
        gene_anno_dic[gene_name]= anno_name.strip()


diapause_rn = diapause.rename(columns={"Gene2": "Correlation"})
diapause_rn = diapause_rn.rename(columns={"Gene1": "Gene2"})
diapause_rn = diapause_rn.rename(columns={"Row": "Gene1"})

def check_annotation(name):
    return gene_anno_dic.get(name, name)

diapause_rn["Annotation_gene1"] = diapause_rn["Gene1"].apply(check_annotation)
diapause_rn["Annotation_gene2"] = diapause_rn["Gene2"].apply(check_annotation)

diapause_rn["Correlation"].fillna(1, inplace=True)

# print(diapause_rn)

genes = diapause_rn["Gene1"].unique()
# df_filtered = diapause_rn[diapause_rn["Correlation"] >= 0.1]
# diapause_rn["Correlation"] += 1
df_filtered = diapause_rn
gene_dictionary = {}
to_remove = ["tim","9062at7088","per","TIMELESS","CLOCK"]
for row in df_filtered.itertuples():
    if (row.Annotation_gene1 not in to_remove) and (row.Annotation_gene2 not in to_remove):
        if row.Annotation_gene1 not in gene_dictionary:
            gene_dictionary[row.Annotation_gene1] = {}
        if row.Annotation_gene1 != row.Annotation_gene2:
            gene_dictionary[row.Annotation_gene1][row.Annotation_gene2] = row.Correlation

gene_dict = gene_dictionary
print("done")


done


In [8]:
import networkx as nx
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform

genes = list(gene_dict.keys())

# Initialize a similarity matrix
n = len(genes)
similarity_matrix = np.zeros((n, n))



for i, gene1 in enumerate(genes):
    for j, gene2 in enumerate(genes):
        if gene2 in gene_dict[gene1]:
            similarity_matrix[i, j] = round(gene_dict[gene1][gene2],2)
            similarity_matrix[j, i] = round(gene_dict[gene1][gene2],2)  # Ensure symmetry
        else:
            similarity_matrix[i, j] = 0  # No direct similarity (or self-similarity)
            similarity_matrix[j, i] = 0


min_similarity = np.min(similarity_matrix)
print(min_similarity)

# Convert similarity matrix to a distance matrix
distance_matrix = 1 - similarity_matrix

np.fill_diagonal(distance_matrix, 0)

# Flatten the distance matrix to a condensed form (required by scipy)
condensed_distance_matrix = squareform(distance_matrix)




-0.74


In [9]:
def average_similarity(cluster, similarity_matrix, genes):
    """
    Compute the average similarity within a cluster.
    """
    if len(cluster) < 2:
        return 0.0  # No similarity for single-gene clusters
    
    # Get indices of genes in the cluster
    indices = [genes.index(gene) for gene in cluster]
    
    # Extract the submatrix for the cluster
    submatrix = similarity_matrix[np.ix_(indices, indices)]
    
    # Compute the average similarity (excluding diagonal)
    return np.sum(submatrix) / (len(cluster) * (len(cluster) - 1))


In [12]:
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.ERC_new_diapause_nondia/3.For Clusturing"
output = []
with open(f"{location}/clusters_diapause.txt", 'r') as cluster_file:
    cluster_lines = cluster_file.readlines()
output = cluster_lines[0].strip()+"\tNonDiapause"
for lines in cluster_lines[1:]:
    lines_split = lines.strip().split("\t")
    cluster = lines_split[1].split(",")
    output += f"\n{lines.strip()}\t{round(average_similarity(cluster, similarity_matrix, genes),2)}"
    # break
# print(output)

with open(f"{location}/clusters_nondiapause.txt", 'w') as out_file:
    out_file.write(output)